In [1]:
import numpy as np

In [2]:
cd drive/MyDrive/머신러닝 실습/

/content/drive/MyDrive/머신러닝 실습


In [ ]:
data = open('text.txt', 'r').read()
vocabs = list(set(data))
data_size = len(data)
vocab_size = len(vocabs)

print(data_size, vocab_size)

ch_to_idx = {ch: idx for idx, ch in enumerate(vocabs)}
idx_to_ch = {idx: ch for idx, ch in enumerate(vocabs)}

In [8]:
# hyperparameter
hidden_size = 100
seq_length = 25
learning_rate = 0.1

# model parameter
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(vocab_size, hidden_size) * 0.01
bh = np.zeros([hidden_size, 1])
by = np.zeros([vocab_size, 1])

In [9]:
def softmax(x):
  return np.exp(x) / np.sum(np.exp(x))

In [10]:
def loss_func(inputs, targets, hprev):
  
  # forward
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = hprev.copy()
  loss = 0

  for t in range(len(inputs)):
    xs[t] = np.zeros([vocab_size, 1])
    xs[t][inputs[t]] = 1

    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + bh + np.dot(Whh, hs[t-1]))
    ys[t] = np.dot(Why, hs[t]) + by
    ps[t] = softmax(ys[t])

    hypothesis = ps[t][targets[t]][0]
    loss += -np.log(hypothesis)

  
  # backward
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])

  for t in reversed(range(len(inputs))):
    dy = ps[t].copy()
    dy[targets[t]] -= 1

    dby += dy
    dWhy += np.dot(dy, hs[t].T)
    dh = np.dot(Why.T, dy) + dhnext

    dhraw = (1 - hs[t] * hs[t]) * dh

    dbh += dhraw
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
    dWxh += np.dot(dhraw, xs[t].T)

  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam)

  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [11]:
def sample(seed_idx, n, h):
  x = np.zeros([vocab_size, 1])
  x[seed_idx] = 1

  idxes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + bh + np.dot(Whh, h))
    y = np.dot(Why, h) + by
    p = softmax(y)

    idx = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros([vocab_size, 1])
    x[idx] = 1
    idxes.append(idx)

  return idxes

In [12]:
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0 / vocab_size) * seq_length

In [13]:
iter = 0
data_ptr = 0

while True:
  if iter == 0 or data_ptr+seq_length+1 >= data_size:
    hprev = np.zeros([hidden_size, 1])
    data_ptr = 0

  inputs = [ch_to_idx[ch] for ch in data[data_ptr:data_ptr+seq_length]]
  targets = [ch_to_idx[ch] for ch in data[data_ptr+1:data_ptr+seq_length+1]]

  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_func(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if iter % 500 == 0:
    print('iter: ', iter, 'loss: ', smooth_loss)

  # sample
  if iter % 500 == 0:
    sample_idxes = sample(inputs[0], 200, hprev)
    text = ''.join(idx_to_ch[idx] for idx in sample_idxes)
    print('==========\n', text, '===========\n')

  # 파라미터 update (Adagrad)
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

  iter += 1
  data_ptr += seq_length